# Multi Resolution Simulation

### Classes and modules

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
import datetime
from IPython.display import display
import copy

#For plotting
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.rcParams["lines.color"] = "w"
plt.rcParams["text.color"] = "w"
plt.rcParams["axes.labelcolor"] = "w"
plt.rcParams["xtick.color"] = "w"
plt.rcParams["ytick.color"] = "w"

GPU Ocean-modules:

In [ ]:
from gpuocean.ensembles import MultiResOceanStateEnsemble
from gpuocean.utils import IPythonMagic, NetCDFInitialization

In [ ]:
%cuda_context_handler gpu_ctx

In [ ]:
%cuda_context_handler gpu_ctx_refined

## Selecting loc

As initial and boundary conditions to the simulation, we use data from the Norkyst800 s-level files:

In [ ]:
nk800_url  = ["../../../havvarsel-data/ocean_his.an.20190716.nc"]

data_args = NetCDFInitialization.removeMetadata(NetCDFInitialization.getInitialConditionsNorKystCases(nk800_url, "lofoten", download_data=False, norkyst_data=False))
data_args.keys()

In [ ]:
sim_args = {
    "dt": 0.0,
    "write_netcdf":False,
    "model_time_step": 60
     }

## Simulation

In [ ]:
from gpuocean.SWEsimulators import CDKLM16

sim = CDKLM16.CDKLM16(gpu_ctx, **sim_args, **data_args)

### Refined Simulation

Defining local area

In [ ]:
# [[y0,x0],[y1,x1]]
loc = [[100, 100], [200,300]]

In [ ]:
eta, hu, hv = sim.download()

fig, ax = plt.subplots(1,1, figsize=(12,6))
ax.imshow(eta, origin="lower", vmin=-1, vmax=1, cmap="coolwarm")

import matplotlib.patches as patches
area = patches.Rectangle((loc[0][0], loc[0][1]), loc[1][1]-loc[0][1], loc[1][0]-loc[0][0], edgecolor='black', facecolor='none', lw=3)
ax.add_patch(area)

Extract local IC

In [ ]:
sim.give_birth(gpu_ctx_refined, loc, 1.2)

In [ ]:
eta, hu, hv = sim.child.download()

fig, ax = plt.subplots(1,1, figsize=(12,6))
ax.imshow(eta, origin="lower", vmin=-1, vmax=1, cmap="coolwarm")

In [ ]:
sim.child.step(60)

### Time-Stepping

First model time step

In [ ]:
eta, hu, hv = sim.download()
t0 = sim.t
bc_data_north_t0 = [x[loc[1][0]+1, loc[0][1]:loc[1][1]] for x in [eta, hu, hv]]
bc_data_south_t0 = [x[loc[0][0]-1, loc[0][1]:loc[1][1]] for x in [eta, hu, hv]]
bc_data_west_t0 = [x[loc[0][0]:loc[1][0], loc[0][1]-1] for x in [eta, hu, hv]]
bc_data_east_t0 = [x[loc[0][0]:loc[1][0], loc[1][1]+1] for x in [eta, hu, hv]]

In [ ]:
sim.step(sim.model_time_step)

In [ ]:
eta, hu, hv = sim.download()
t1 = sim.t
bc_data_north_t1 = [x[loc[1][0]+1, loc[0][1]:loc[1][1]] for x in [eta, hu, hv]]
bc_data_south_t1 = [x[loc[0][0]-1, loc[0][1]:loc[1][1]] for x in [eta, hu, hv]]
bc_data_west_t1 = [x[loc[0][0]:loc[1][0], loc[0][1]-1] for x in [eta, hu, hv]]
bc_data_east_t1 = [x[loc[0][0]:loc[1][0], loc[1][1]+1] for x in [eta, hu, hv]]

In [ ]:
from gpuocean.utils import Common
t = [t0, t1]
for cardinal in ["north", "south", "east", "west"]:
    exec("{0} = Common.SingleBoundaryConditionData(h = [bc_data_{0}_t0[0], bc_data_{0}_t1[0]],\
                                            hu = [bc_data_{0}_t0[1], bc_data_{0}_t1[1]],\
                                            hv = [bc_data_{0}_t0[2], bc_data_{0}_t1[2]])".format(cardinal))
bc_data = Common.BoundaryConditionsData(t=t, north=north, south=south, east=east, west=west)

In [ ]:
sim.child.bc_kernel = Common.BoundaryConditionsArakawaA(gpu_ctx_refined, sim.child.nx, sim.child.ny, 2, 2, \
                                                           sim.child.boundary_conditions, bc_data)


In [ ]:
sim.child.step(sim.model_time_step)

Repeated model time steps

In [ ]:
while sim.t < 3600:
    eta, hu, hv = sim.download()
    t0 = sim.t
    bc_data_north_t0 = [x[loc[1][0]+1, loc[0][1]:loc[1][1]] for x in [eta, hu, hv]]
    bc_data_south_t0 = [x[loc[0][0]-1, loc[0][1]:loc[1][1]] for x in [eta, hu, hv]]
    bc_data_west_t0 = [x[loc[0][0]:loc[1][0], loc[0][1]-1] for x in [eta, hu, hv]]
    bc_data_east_t0 = [x[loc[0][0]:loc[1][0], loc[1][1]+1] for x in [eta, hu, hv]]

    sim.step(sim.model_time_step)

    eta, hu, hv = sim.download()
    t1 = sim.t
    bc_data_north_t1 = [x[loc[1][0]+1, loc[0][1]:loc[1][1]] for x in [eta, hu, hv]]
    bc_data_south_t1 = [x[loc[0][0]-1, loc[0][1]:loc[1][1]] for x in [eta, hu, hv]]
    bc_data_west_t1 = [x[loc[0][0]:loc[1][0], loc[0][1]-1] for x in [eta, hu, hv]]
    bc_data_east_t1 = [x[loc[0][0]:loc[1][0], loc[1][1]+1] for x in [eta, hu, hv]]

    t = [t0, t1]
    for cardinal in ["north", "south", "east", "west"]:
        exec("{0} = Common.SingleBoundaryConditionData(h = [bc_data_{0}_t0[0], bc_data_{0}_t1[0]],\
                                                hu = [bc_data_{0}_t0[1], bc_data_{0}_t1[1]],\
                                                hv = [bc_data_{0}_t0[2], bc_data_{0}_t1[2]])".format(cardinal))
    bc_data = Common.BoundaryConditionsData(t=t, north=north, south=south, east=east, west=west)

    sim.child.bc_kernel = Common.BoundaryConditionsArakawaA(gpu_ctx_refined, sim.child.nx, sim.child.ny, 2, 2, \
                                                           sim.child.boundary_conditions, bc_data)

    sim.child.step(sim.model_time_step)

In [ ]:
eta, hu, hv = sim.download()
eta_refined, hu_refined, hv_refined = sim.child.download()

In [ ]:
fig, axs = plt.subplots(2,3,figsize=(12,6))

axs[0,0].imshow(eta[loc[0][0]:loc[1][0],loc[0][1]:loc[1][1]], origin="lower", vmin=-1, vmax=1, cmap="coolwarm")
axs[0,0].set_title("eta (coarse)")
axs[0,1].imshow(hu[loc[0][0]:loc[1][0],loc[0][1]:loc[1][1]], origin="lower", vmin=-100, vmax=100, cmap="coolwarm")
axs[0,1].set_title("hu (coarse)")
axs[0,2].imshow(hv[loc[0][0]:loc[1][0],loc[0][1]:loc[1][1]], origin="lower", vmin=-100, vmax=100, cmap="coolwarm")
axs[0,2].set_title("hv (coarse)")

axs[1,0].imshow(eta_refined, origin="lower", vmin=-1, vmax=1, cmap="coolwarm")
axs[1,0].set_title("eta (refined)")
axs[1,1].imshow(hu_refined, origin="lower", vmin=-100, vmax=100, cmap="coolwarm")
axs[1,1].set_title("hu (refined)")
axs[1,2].imshow(hv_refined, origin="lower", vmin=-100, vmax=100, cmap="coolwarm")
axs[1,2].set_title("hv (refined)")

for ax_s in axs:
    for ax in ax_s:
        ax.get_xaxis().set_ticks([])
        ax.get_yaxis().set_ticks([])

In [ ]:
eta, hu, hv = sim_refined.download()

fig, ax = plt.subplots(1,1, figsize=(12,6))
ax.imshow(eta, origin="lower", vmin=-1, vmax=1, cmap="coolwarm")